# **LinkedIn Post Retrieval**

This Python script leverages the LinkedIn API to fetch posts from a  Showcase Page. The script is thoughtfully designed to manage pagination efficiently, ensuring the retrieval of all available posts through successive GET requests. By parsing the LinkedIn API responses, the script extracts crucial details such as post content and creation time. The extracted information is then structured into a Pandas DataFrame, presenting a tabular format that facilitates straightforward analysis.


Key components of the script include:


**Access Token and API Endpoint:** Users must replace placeholders with their actual LinkedIn API access token and Showcase Page ID. These credentials are vital for authentication and specifying the target organization or Showcase Page.

**API Request:** The script orchestrates a series of GET requests to the LinkedIn API, navigating through paginated responses to compile a comprehensive set of posts.

**Data Extraction:** Response data undergoes parsing to selectively extract pertinent details, including post content and creation time, from the intricate nested structure of the API response.

**DataFrame Creation:** The extracted data is meticulously organized into a Pandas DataFrame, featuring distinct columns for 'Post Content' and 'Creation Time'.

**Print Output:** The script culminates in printing the resulting DataFrame to the console, furnishing a neatly formatted tabular view of the post data. Additionally, the total number of retrieved posts is displayed.


In essence, this script establishes a robust foundation for the exploration and analysis of LinkedIn posts, offering a systematic and structured approach for users keen on deriving insights from their LinkedIn data.






In [ ]:
import requests
import pandas as pd

# Replace 'YOUR_ACCESS_TOKEN' with the actual access token obtained through LinkedIn's API authentication process
access_token = 'YOUR_ACCESS_TOKEN'

# Replace 'SHOWCASE_PAGE_ID' with the actual Showcase Page ID
showcase_page_id = 'SHOWCASE_PAGE_ID'

# Define the API endpoint for retrieving posts for a specific organization or Showcase Page
api_url = f'API endpoint:{showcase_page_id}'

# Set up the headers with the access token
headers = {
    'Authorization': f'Bearer {access_token}',
}

try:
    # Initialize variables
    all_posts = []
    start = 0
    count = 10  # You can adjust the count based on the desired number of posts per request

    while True:
        # Make a GET request to the LinkedIn API to retrieve posts
        params = {'start': start, 'count': count}
        response = requests.get(api_url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse and process the response data to get post information
            posts_data = response.json().get('elements', [])

            # Add the posts to the list
            all_posts.extend(posts_data)

            # Check if there are more pages and update the start value
            pagination = response.json().get('paging', {})
            start = pagination.get('start', 0) + count

            if start >= pagination.get('total', 0):
                break  # Exit the loop if all posts have been retrieved

        else:
            print(f"Error fetching posts: {response.status_code} - {response.text}")
            break  # Exit the loop in case of an error

    # Create a DataFrame from the extracted data
    data = {'Post Content': [], 'Creation Time': []}
    for post in all_posts:
        # Extract 'text' from nested structure
        post_text = post.get('text', {}).get('text', '')

        # Extract 'time' from nested structure
        creation_date = post.get('created', {}).get('time', '') or post.get('creationDateTime', {}).get('time', '')

        data['Post Content'].append(post_text)
        data['Creation Time'].append(creation_date)

    df = pd.DataFrame(data)

    # Print the DataFrame
    print(df)

    # Print the count of all posts
    print(f"Total number of posts: {len(all_posts)}")

except requests.exceptions.RequestException as e:
    print(f"Request Exception: {e}")
